# Study Query LLM - Google Colab Setup

This notebook sets up and runs the Study Query LLM application in Google Colab.

## Features
- Run LLM inferences across multiple providers (Azure OpenAI, OpenAI, Hyperbolic)
- Automatic logging to SQLite database
- Analytics dashboard with provider comparison
- No local installation required!

## Setup Instructions

1. **Get the source code** - Clone from GitHub or upload the project folder (Step 1)
2. **Set your API keys** - Use Colab Secrets (recommended) or set in code (Step 3)
3. **Run all cells** - Install dependencies and start the app
4. **Use the app** - It will open in a new tab or show a URL

**Important:** Make sure to update the GitHub URL in Step 1 with your repository URL, or upload the project folder to Colab.

Note: The app will run in this Colab session. When you close the notebook, the session ends.


## Step 1: Get the Source Code

**Choose one method below to get the source code:**


In [ ]:
# OPTION 1: Clone from GitHub (Recommended)
# Replace with your repository URL
!git clone https://github.com/yourusername/study-query-llm.git
%cd study-query-llm

# OPTION 2: If you uploaded the project folder to Colab, uncomment:
# %cd /content/study-query-llm  # Adjust path if different

# Install the package and dependencies
%pip install -q -e .
%pip install -q panel python-dotenv openai tenacity sqlalchemy pandas

print("✅ Source code and dependencies installed!")


## Step 2: Verify Installation

Check that the package is installed correctly.


In [ ]:
# Verify the package can be imported
try:
    import study_query_llm
    from study_query_llm.config import config
    print("✅ Package imported successfully!")
    print(f"   Package location: {study_query_llm.__file__}")
except ImportError as e:
    print(f"❌ Error importing package: {e}")
    print("\nTroubleshooting:")
    print("1. Make sure you ran the previous cell (git clone or uploaded files)")
    print("2. Check that you're in the correct directory")
    print("3. Verify the package was installed with 'pip install -e .'")


## Step 3: Configure API Keys

**Recommended:** Use Colab Secrets (left sidebar → 🔑 Secrets)

1. Click the 🔑 icon in the left sidebar
2. Click "+ Add secret"
3. Add secrets with these exact names:
   - `AZURE_OPENAI_API_KEY`
   - `AZURE_OPENAI_ENDPOINT`
   - `AZURE_OPENAI_DEPLOYMENT`
   - `AZURE_OPENAI_API_VERSION`
   - (Optional) `OPENAI_API_KEY`, `OPENAI_MODEL`
   - (Optional) `HYPERBOLIC_API_KEY`, `HYPERBOLIC_ENDPOINT`

**Alternative:** Set API keys directly in the code cell below (less secure)


In [ ]:
import os

# Try to load from Colab Secrets (recommended method)
try:
    from google.colab import userdata
    
    # Load secrets from Colab Secrets (left sidebar)
    secrets_loaded = False
    try:
        os.environ["AZURE_OPENAI_API_KEY"] = userdata.get('AZURE_OPENAI_API_KEY')
        os.environ["AZURE_OPENAI_ENDPOINT"] = userdata.get('AZURE_OPENAI_ENDPOINT')
        os.environ["AZURE_OPENAI_DEPLOYMENT"] = userdata.get('AZURE_OPENAI_DEPLOYMENT', 'gpt-4o')
        os.environ["AZURE_OPENAI_API_VERSION"] = userdata.get('AZURE_OPENAI_API_VERSION', '2024-02-15-preview')
        
        # Optional providers
        try:
            os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
            os.environ["OPENAI_MODEL"] = userdata.get('OPENAI_MODEL', 'gpt-4')
        except:
            pass  # OpenAI not configured
        
        try:
            os.environ["HYPERBOLIC_API_KEY"] = userdata.get('HYPERBOLIC_API_KEY')
            os.environ["HYPERBOLIC_ENDPOINT"] = userdata.get('HYPERBOLIC_ENDPOINT', 'https://api.hyperbolic.xyz')
        except:
            pass  # Hyperbolic not configured
        
        secrets_loaded = True
        print("✅ Configuration loaded from Colab Secrets!")
        
    except Exception as e:
        print(f"⚠️  Could not load all secrets from Colab Secrets: {e}")
        print("   Falling back to environment variables...")
        secrets_loaded = False
        
except ImportError:
    # Not running in Colab, use environment variables
    secrets_loaded = False
    print("ℹ️  Not running in Colab - using environment variables")

# Fallback: Set API keys directly (if not using Colab Secrets)
if not secrets_loaded:
    # Azure OpenAI
    os.environ.setdefault("AZURE_OPENAI_API_KEY", "your-azure-api-key-here")
    os.environ.setdefault("AZURE_OPENAI_ENDPOINT", "https://your-resource.openai.azure.com/")
    os.environ.setdefault("AZURE_OPENAI_DEPLOYMENT", "gpt-4o")
    os.environ.setdefault("AZURE_OPENAI_API_VERSION", "2024-02-15-preview")
    
    # OpenAI (optional)
    # os.environ.setdefault("OPENAI_API_KEY", "your-openai-api-key-here")
    # os.environ.setdefault("OPENAI_MODEL", "gpt-4")
    
    # Hyperbolic (optional)
    # os.environ.setdefault("HYPERBOLIC_API_KEY", "your-hyperbolic-api-key-here")
    # os.environ.setdefault("HYPERBOLIC_ENDPOINT", "https://api.hyperbolic.xyz")
    
    if os.environ["AZURE_OPENAI_API_KEY"] == "your-azure-api-key-here":
        print("⚠️  Using default placeholder values!")
        print("   Please set your API keys in Colab Secrets or update the code above.")

# Database (SQLite - will be created automatically)
os.environ["DATABASE_URL"] = "sqlite:///study_query_llm.db"

print("\n✅ Configuration complete!")


## Step 4: Initialize Database


In [ ]:
# Initialize the database
from study_query_llm.db.connection import DatabaseConnection
from study_query_llm.config import config

db = DatabaseConnection(config.database.connection_string)
db.init_db()

print("✅ Database initialized!")


## Step 5: Start the Application


In [ ]:
# Import and create the app
from panel_app.app import serve_app
from IPython.display import Markdown, display

# Stop any existing server
if 'dashboard_server' in globals():
    try:
        dashboard_server.stop()
    except Exception:
        pass

# Start the server (Colab will create a public URL)
dashboard_server, dashboard_url = serve_app(
    address='0.0.0.0',  # Listen on all interfaces for Colab
    port=5006,
    route=None,
    open_browser=False,
)

# Display the URL
display(Markdown(f"## ✅ Application Started!\n\n**[Open the dashboard]({dashboard_url})**\n\nOr copy this URL: `{dashboard_url}`"))


## Alternative: Display in Notebook

If the above doesn't work, try displaying the app directly in the notebook:


In [ ]:
# Alternative: Display app in notebook cell
# Uncomment the line below to display the app inline
# app


## Troubleshooting

### If the app doesn't start:
1. Check that all API keys are set correctly
2. Verify your Azure deployment name matches what's in Azure Portal
3. Make sure all cells above have run successfully
4. Ensure the project source code is accessible (uploaded or cloned)

### To stop the app:
- Interrupt the kernel (Runtime → Interrupt execution)
- Or restart the runtime (Runtime → Restart runtime)

### Database location:
- The SQLite database is created in the Colab session
- It will be deleted when the session ends
- To persist data, download the database file or use a cloud database

## Next Steps

1. Go to the **Inference** tab in the app
2. Select your provider (Azure, OpenAI, etc.)
3. For Azure: Click "Load Deployments" and select a deployment
4. Enter a prompt and run inference
5. Check the **Analytics** tab to see your results!
